# FPCUP Demo
Author: Olivier Burggraaff, Universiteit Leiden, 2024

This notebook demonstrates some of the functionality of the Python package developed within the FPCUP project.

## Python / Jupyter instructions
To do:
Instructions on using Python.
Instructions on using Jupyter.
Instructions on installing FPCUP package.

In [ ]:
import fpcup

## Data

### BRP
You can import data from the 2019–2024 BRPs (Basisregistratie Percelen) as follows:

In [ ]:
# Setup
brp_filename = fpcup.io.Path("data/brp/brp2020.gpkg")
brp = fpcup.io.read_geodataframe(brp_filename)

# Example output
brp

The BRP contains geographical data, which we can use to draw figures.
For example, the following cell will generate a figure showing the locations of all plots where barley, maize, sorghum, soy, or wheat was grown in the selected year.

In [ ]:
# Plot
fpcup.plotting.brp_map(brp, column="crop_species", title="Selected crop types", colour_dict=fpcup.plotting.brp_crops_colours)

Using the `province` keyword argument, we can also select only one area, for example:

In [ ]:
# Plot
province = fpcup.geo.process_input_province("Zuid-Holland") 
fpcup.plotting.brp_map(brp, province=province, column="crop_species", title="Selected crop types", colour_dict=fpcup.plotting.brp_crops_colours)

### Running WOFOST
WOFOST (WOrld FOod STudies) is a crop model developed at Wageningen University & Research.
The FPCUP package provides an easy-to-use interface for running simulations in WOFOST (using its PCSE implementation).

### WOFOST ensembles